In [201]:
import pandas as pd
import numpy as np
import h5py
from IPython.display import clear_output

In [205]:
TRAIN_FILE_PATH = "/sci/nosnap/michall/roeizucker/jupyter_notebooks/Tom_Hope_Project/spliceAI_data/datafile_train_all.h5"
TEST_FILE_PATH = "/sci/nosnap/michall/roeizucker/jupyter_notebooks/Tom_Hope_Project/spliceAI_data/datafile_test_all.h5"
TRAIN_DF_PATH = "/sci/nosnap/michall/roeizucker/token_classification/spliceai_train.csv"
TEST_DF_PATH = "/sci/nosnap/michall/roeizucker/token_classification/spliceai_test.csv"
START_LABEL = "start"
END_LABEL = "end"
NONE_LABEL = "none"
COLUMNS = ["sequnces","labels","name","pos_start","pos_end","chrom","description"]

In [198]:
general_data_dict = {"df_train_path" : TRAIN_DF_PATH, "df_test_path":TEST_DF_PATH, "data_type": "splice_AI","encoding":"hg19"}

In [212]:
def create_df(filename):
    with h5py.File(filename, "r") as f:
        all_vals = []
        # print("length",len(f["NAME"]))
        file_length = len(f["NAME"])
        # file_length = 5000
        # for index in range(len(f["NAME"])):
        for index in range(file_length):
            if index % 100 == 0:
                clear_output(wait=True)
                print(filename.split("/")[-1])
                print(str(int(100*(index / file_length))) + "%")
                
            seq = f["SEQ"][index].decode().upper()
            name = f["NAME"][index].decode()
            chrom = f["CHROM"][index].decode()
            strand = f["STRAND"][index].decode()
            paralog = f["PARALOG"][index]
            
            jn_start_raw = f["JN_START"][index]
            jn_end_raw = f["JN_END"][index]
            tx_start = int(f["TX_START"][index].decode())
            tx_end = int(f["TX_END"][index].decode())
            length = tx_end-tx_start
            jn_end = np.array(jn_end_raw[0].decode().split(",")[:-1],dtype=int) - tx_start
            jn_start = np.array(jn_start_raw[0].decode().split(",")[:-1],dtype=int) - tx_start
            
            jn_start = list(zip(jn_start,[START_LABEL]*len(jn_start)))
            jn_end = list(zip(jn_end,[END_LABEL]*len(jn_start)))
            all_pos = sorted(jn_start + jn_end)
    
            sequences = []
            labels = []
            
            last_pos = 0
            for pos,pos_type in all_pos:
                sequences.append(seq[last_pos:pos])
                labels.append(NONE_LABEL)
                sequences.append(seq[pos:pos+1])
                labels.append(pos_type)
                last_pos = pos+1
            sequences.append(seq[last_pos:length])
            labels.append(NONE_LABEL)
            # all_vals.append([sequences,labels,name,strand,chrom,"paralog: " + str(paralog)])
            all_vals.append([sequences,labels,name,tx_start,tx_end,chrom,{"paralog":str(paralog), "strand":strand}])
            # print(length)
    df = pd.DataFrame(all_vals,columns=COLUMNS)
    return df
            # print()
    # return all_vals
        
train_df = create_df(TRAIN_FILE_PATH)
test_df = create_df(TEST_FILE_PATH)

datafile_test_all.h5
98%


In [216]:
train_df.to_csv(TRAIN_DF_PATH)
test_df.to_csv(TEST_DF_PATH)